1. Write a decorator that ensures a function is only called by users with a
 specific role. Each function should have an user_type with a string 
type in kwargs.

In [144]:
import functools

def is_admin(func):
    @functools.wraps(func) # Preserve the function metadata
    def wrapper(*args, **kwargs):
        user_type = kwargs.get('user_type')
        if user_type == 'admin':
            return func(*args, **kwargs)
        else:
            raise ValueError('Permission denied')
    return wrapper


In [142]:
@is_admin
def show_customer_receipt(user_type: str):
    # Some very dangerous operation
    return('Hello World!')

result = show_customer_receipt(user_type='admin')
print(result)

try:
    result = show_customer_receipt(user_type='user')
    print(result)
except ValueError as e:
    print(e)


Hello World!
Permission denied


2. Write a decorator that wraps a function in a try-except block and prints an error if any type of error has happened.

In [181]:
def catch_errors(func):
    @functools.wraps(func) # Preserve the function metadata
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Found 1 error during execution of your function: {e.__class__.__name__} - {e}")
    return wrapper

In [182]:
@catch_errors
def some_function_with_risky_operation(data):
    print(data['key'])


some_function_with_risky_operation({'foo': 'bar'})
# Found 1 error during execution of your function: KeyError no such key as foo

some_function_with_risky_operation({'key': 'bar'})
# bar

Found 1 error during execution of your function: KeyError - 'key'
bar


3. Optional: Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations. It should work for all possible functions. Don`t forget to check the return type as well.

In [200]:
from typing import Callable, Any, Type, get_type_hints

def check_types(func: Callable) -> Callable:
    @functools.wraps(func) # Preserve the function metadata
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        # Get the type hints from the function annotations
        hints = get_type_hints(func)
        
        # Check argument types
        for arg_name, arg_value in zip(func.__code__.co_varnames, args):
            if arg_name in hints:
                expected_type = hints[arg_name]
                if not isinstance(arg_value, expected_type):
                    raise TypeError(f"Argument {arg_name} must be {expected_type.__name__}, not {type(arg_value).__name__}")
        
        # Call the original function
        result = func(*args, **kwargs)
        
        # Check return type
        if 'return' in hints:
            expected_return_type = hints['return']
            if not isinstance(result, expected_return_type):
                raise TypeError(f"Return value must be {expected_return_type.__name__}, not {type(result).__name__}")
        
        return result
    
    return wrapper

In [208]:
@check_types
def add(a: int, b: int) -> int:
    return a + b
    # return str(a + b) # To check return type

add(1, 2)
# 3

add("1", "2")
# TypeError: Argument a must be int, not str

assert add(1, 2) == '3'

TypeError: Argument a must be int, not str

4. Optional. Create a function that caches the result of a function, so that if it is called with the same argument multiple times, it returns the cached result first instead of re-executing the function.

In [211]:
def cache_decorator(func):
    cache = {}

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = (args, tuple(kwargs.items()))
        
        if key in cache:
            return cache[key]
        else:
            result = func(*args, **kwargs)
            cache[key] = result
            return result
    
    return wrapper

@cache_decorator
def expensive_function(x, y):
    print("Executing expensive_function...")
    return x + y

result1 = expensive_function(2, 3)
print("Result 1:", result1)

result2 = expensive_function(2, 3)  # This time, the result will be cached
print("Result 2:", result2)

result3 = expensive_function(4, 5)  # This will trigger the execution again
print("Result 3:", result3)


Executing expensive_function...
Result 1: 5
Result 2: 5
Executing expensive_function...
Result 3: 9


5. Optional. Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute

In [210]:
import time

def rate_limiter(max_calls, interval):
    """
    A decorator to limit the rate at which a function can be called.
    
    Args:
        max_calls (int): Maximum number of calls allowed within the specified interval.
        interval (float): Time interval in seconds.
    """
    def decorator(func):
        last_called = [0]
        call_count = [0]

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()
            elapsed_time = current_time - last_called[0]
            
            if elapsed_time < interval:
                if call_count[0] >= max_calls:
                    wait_time = interval - elapsed_time
                    time.sleep(wait_time)
                    last_called[0] = current_time
                    call_count[0] = 1
                else:
                    call_count[0] += 1
            else:
                last_called[0] = current_time
                call_count[0] = 1
            
            return func(*args, **kwargs)
        
        return wrapper
    return decorator


@rate_limiter(max_calls=5, interval=60)  # Allow 5 calls per minute
def my_function():
    print("Function called")

# Test the rate-limited function
for i in range(10):
    my_function()
    time.sleep(5)  # Simulate some time between function calls


Function called
Function called
Function called
Function called
Function called


KeyboardInterrupt: 